In [1]:
import pandas as pd
import re
import string
import logging
import json
from textwrap import indent
from typing import Dict, List, Tuple
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim.downloader as api
import numpy as np

model = api.load("glove-twitter-100")

In [2]:
vec_len = 100

def cleantext(text_str: str) -> str:
    #will replace the html characters with " "
    text_str=re.sub('<.*?>', ' ', text_str)  
    text_str = text_str.translate(str.maketrans(' ',' ',string.punctuation)) # remove punctuations
    text_str = re.sub('[^a-zA-Z]',' ',text_str) # Only alphabets
    text_str = re.sub("\n"," ",text_str)
    text_str = re.sub(' +', ' ', text_str) # remove multiple spaces
    text_str = text_str.lower()
    # will split and join the words
    text_str = text_str.split()
    return text_str

def wordvecs(tweet: str) -> np.array:
    word_array = cleantext(tweet)
    sentence_vector = []
    for x in word_array:
        try:
            row_vec = model[x]
        except:
            row_vec = np.zeros(vec_len)
        assert row_vec.shape[0] == vec_len, "Word vecs not consistent"
        sentence_vector.append(row_vec.reshape(-1,vec_len))
        
        sentence_np = np.mean(sentence_vector, axis=0)    
    return sentence_np



In [3]:
df = pd.read_csv('../data/train.csv')
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [8]:
dd = pd.DataFrame(df['tweet'].apply(wordvecs).apply(lambda x: x[0]).values.tolist())

In [9]:
dd.head(2)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.248162,-0.121376,0.299926,0.189048,0.115270,0.158885,0.046286,0.133732,0.140999,-0.215381,...,0.104440,-0.216360,0.080718,0.29589,0.125022,-0.007140,-0.054420,-0.176437,0.190849,0.045643
1,0.017812,0.149181,-0.142397,0.165627,0.059389,-0.141543,0.179981,0.074654,0.217905,0.136639,...,-0.107918,0.093294,0.012745,-0.19452,0.159313,-0.218376,-0.250479,-0.152292,0.116835,0.214693
